<img src=".\images\Logo.png" width=150 align="left" /> <img src=".\images\Logo2.jpg" width=450 align="right" />


# <center><font color= #1e8449 > <b>CU18_Infraestructuras_eventos</font></center>

<font color='steelblue'><b>Citizenlab Data Science Methodology > III - Feature Engineering Domain </font>
***
> # <font color='steelblue'> <b>14.- Feature Data Transform</font>

<font color='steelblue'>Feature Data Transform is the process that allows change (if is required) the type and/or distribution of data features (e.g. scaling, normalizing o standardizing data features).</font>


## <font color='steelblue'>Tasks</font>

<font color='steelblue'><b>Perform Basic Data Transforms						
    
<font color='steelblue'><b>Perform Categorical Variable Transformation						
- Encode Transformation					
    - One-hot encoding				
    - Ordinal encoding				
    - Dummy encoding				
- Evaluate a Logistic Regression model					
- Consider Embedding if text mining context					
    
<font color='steelblue'><b>Perform Numeric Variable Transformation						
- Scale Transformation					
    - Normalization				
    - Standardization				
    - IQR Robust Scaler Transform				
    - Evaluate a KNN model				
- Distribution Transformation					
    - Discretization				
        - Uniform			
        - Clustered(k-Means)			
        - Quantile			
            - Normal Quantile		
            - Uniform Quantile		
            - Evaluate a KNN model		
        - Evaluate a KNN model			
    - Power transforms (Make Distributions More Gaussian)				
        - Box-Cox Transform			
        - Yeo-Johnson Transform			
        - Evaluate a KNN model			


## Consideraciones casos CitizenLab programados en R

* Algunas de las tareas de este proceso se han realizado en los notebooks del proceso 05 Data Collection porque eran necesarias para las tareas ETL. En esos casos, en este notebook se referencia al notebook del proceso 05 correspondiente
* Otras tareas típicas de este proceso se realizan en los notebooks del dominio IV al ser más eficiente realizarlas en el propio pipeline de modelización.
* Por tanto en los notebooks de este proceso de manera general se incluyen las comprobaciones necesarias, y comentarios si procede
* Las tareas del proceso se van a aplicar solo a los archivos que forman parte del despliegue, ya que hay muchos archivos intermedios que no procede pasar por este proceso
* El nombre de archivo del notebook hace referencia al nombre de archivo del proceso 05 al que se aplica este proceso, por eso pueden no ser correlativa la numeración
* Las comprobaciones se van a realizar teniendo en cuenta que el lenguaje utilizado en el despliegue de este caso es R

## <font color='green'>File</font>

- <font color='steelblue'> <b>Input File: CU_18_09.3_20_diario_infra</font>
- <font color='steelblue'> <b>Output File: No aplica</font>

### Encoding

Con la siguiente expresión se evitan problemas con el encoding al ejecutar el notebook. Es posible que deba ser eliminada o adaptada a la máquina en la que se ejecute el código.

In [1]:
Sys.setlocale(category = "LC_ALL", locale = "es_ES.UTF-8")

[1] "LC_CTYPE=es_ES.UTF-8;LC_NUMERIC=C;LC_TIME=es_ES.UTF-8;LC_COLLATE=es_ES.UTF-8;LC_MONETARY=es_ES.UTF-8;LC_MESSAGES=en_US.UTF-8;LC_PAPER=es_ES.UTF-8;LC_NAME=C;LC_ADDRESS=C;LC_TELEPHONE=C;LC_MEASUREMENT=es_ES.UTF-8;LC_IDENTIFICATION=C"

## <font color='green'>Settings</font>

### Libraries to use

In [2]:
library(readr)
library(dplyr)
library(tidyr)
library(forcats)
library(lubridate)



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




### Paths

In [3]:
iPath <- "Data/Input/"
oPath <- "Data/Output/"

## <font color='green'>Data Load</font>

<font color='tomato'><b> OPCION A:</b> Seleccionar fichero en ventana para mayor comodidad</font> 

Data load using the {tcltk} package. Ucomment the line if using this option

In [4]:
# file_data <- tcltk::tk_choose.files(multi = FALSE)

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>



In [5]:
iFile <- "CU_18_09.3_20_diario_infra.csv"
file_data <- paste0(iPath, iFile)

if(file.exists(file_data)){
    cat("Se leerán datos del archivo: ", file_data)
} else{
    warning("Cuidado: el archivo no existe.")
}


Se leerán datos del archivo:  Data/Input/CU_18_09.3_20_diario_infra.csv

#### Data file to dataframe

Usar la función adecuada según el formato de entrada (xlsx, csv, json, ...)

In [6]:
data <- read_csv(file_data)


Rows: 377727 Columns: 10
── Column specification ──────────────────────────────────────────────────────────────────
Delimiter: ","
dbl  (9): id_inf, capacidad, demanda, evento_infra, evento_zona, tmed, prec,...
date (1): fecha

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Estructura de  los datos:

In [7]:
data |> glimpse()

Rows: 377,727
Columns: 10
$ id_inf       <dbl> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17…
$ fecha        <date> 2019-01-01, 2019-01-01, 2019-01-01, 2019-01-01, 2019-01-…
$ capacidad    <dbl> 993, 996, 1036, 1020, 992, 1026, 1007, 976, 1037, 972, 94…
$ demanda      <dbl> 883, 888, 922, 1134, 1103, 1139, 897, 1086, 1150, 861, 83…
$ evento_infra <dbl> 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, …
$ evento_zona  <dbl> 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, …
$ tmed         <dbl> 6.953211, 6.196420, 6.483569, 5.875797, 6.212680, 5.87854…
$ prec         <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, …
$ velmedia     <dbl> 0.6433886, 0.3417523, 0.4132169, 0.1820178, 0.2118110, 0.…
$ presMax      <dbl> 952.9357, 950.2191, 950.8051, 951.6768, 953.5118, 952.168…


Muestra de los primeros datos:

In [8]:
data |> slice_head(n = 5)

id_inf,fecha,capacidad,demanda,evento_infra,evento_zona,tmed,prec,velmedia,presMax
<dbl>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2019-01-01,993,883,1,1,6.953211,0,0.6433886,952.9357
2,2019-01-01,996,888,0,0,6.196420,0,0.3417523,950.2191
3,2019-01-01,1036,922,0,0,6.483569,0,0.4132169,950.8051
4,2019-01-01,1020,1134,1,0,5.875797,0,0.1820178,951.6768
5,2019-01-01,992,1103,1,1,6.212680,0,0.2118110,953.5118


## Basic Data Transforms

### Data Selecting

In [9]:
data |> select(1)

id_inf
<dbl>
1
2
3
4
5
6
7
8
9


### Data Filtering

In [12]:
# data |> filter(ano = 2019)

### Insert New Column

In [13]:
data |>
    mutate(x = TRUE)

id_inf,fecha,capacidad,demanda,evento_infra,evento_zona,tmed,prec,velmedia,presMax,x
<dbl>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
1,2019-01-01,993,883,1,1,6.953211,0,0.6433886,952.9357,TRUE
2,2019-01-01,996,888,0,0,6.196420,0,0.3417523,950.2191,TRUE
3,2019-01-01,1036,922,0,0,6.483569,0,0.4132169,950.8051,TRUE
4,2019-01-01,1020,1134,1,0,5.875797,0,0.1820178,951.6768,TRUE
5,2019-01-01,992,1103,1,1,6.212680,0,0.2118110,953.5118,TRUE
6,2019-01-01,1026,1139,0,1,5.878542,0,0.1742736,952.1681,TRUE
7,2019-01-01,1007,897,0,0,6.043628,0,0.2137601,951.9106,TRUE
8,2019-01-01,976,1086,1,0,6.716287,0,0.4128750,953.9489,TRUE
9,2019-01-01,1037,1150,1,1,6.541842,0,0.3536813,951.1342,TRUE


### Delete Column

In [15]:
# data |> select(-x)

### Replace Values

In [ ]:
# data |> mutate(x = case_when(...))

### Rank Data

<font color='darkgrey'><b>Operation</font>

In [ ]:
# data |> mutate(rank = order(n_vacunas))

## Categorical Variable Transformation

### Encode Transformation

In [ ]:
# data |> mutate(x = if_else(x = 'Yes', 1, 0))

#### Ordinal Encoding Transform

In [ ]:
# data |> mutate(x = fct_reorder(...))

#### One Hot Encoding Transform

#### Dummy Variable Encoding Transform

### Embedding Transformation

Specific encode for text mining context. No code here.

## Numeric Variable Transformation: Scale

### Data to Transform

#### Evaluating Normalization Tranform

#### Evaluating Standarization Transform

### Normalization Transform

<font color='tomato'>Select columns</font>

In [ ]:
# vars <- c()

<font color='darkgrey'><b>Operation</font>

### Standarization Transform

<font color='tomato'>Select columns</font>

In [ ]:
# vars <- c()

<font color='darkgrey'><b>Operation</font>

In [ ]:
# data |> mutate(across(vars), scale)

## Numeric Variable Transformation: Distribution

###	Discretization Transform

##### Evaluating Discretization Transformations

#### Uniform Discretization Transform

<font color='tomato'>Select columns</font>

In [ ]:
# retrieve just the numeric input values
# vars <- c(...)

<font color='darkgrey'><b>Operation</font>

In [ ]:
# data |> mutate(across(vars), cut(breaks = 10))

### Power Transform

#### Data to Transform

##### Evaluating Box-Cox tranform

##### Evaluating Yeo-Johnson tranform

#### Box-Cox Transform

<font color='tomato'>Select columns</font>

<font color='darkgrey'><b>Operation</font>

#### Yeo-Johnson Transform

<font color='tomato'>Select columns</font>

<font color='darkgrey'><b>Operation</font>

## <font color='green'>Data Save</font>

* Solo si se han hecho cambios

* No aplica

<font color='tomato'> Identificamos los datos a guardar</font>

In [16]:
data_to_save <- data


<font color='tomato'>Estructura de nombre de archivos:</font>

* Código del caso de uso, por ejemplo "CU_04"
* Número del proceso que lo genera, por ejemplo "_06".
* Resto del nombre del archivo de entrada
* Extensión del archivo

Ejemplo: "CU_04_06_01_01_zonasgeo.json, primer fichero que se genera en la tarea 01 del proceso 05 (Data Collection) para el caso de uso 04 (vacunas) y que se ha transformado en el proceso 06

Importante mantener los guiones bajos antes de proceso, tarea, archivo y nombre

### Proceso 14

In [17]:
caso <- "CU_18"
proceso <- '_014'
tarea <- "_20"
archivo <- ""
proper <- "_diario_infra"
extension <- ".csv"

<font color='tomato'><b> OPCION A:</b> Uso del paquete "tcltk" para mayor comodidad</font>

* Buscar carpeta, escribir nombre de archivo SIN extensión (se especifica en el código)
* Especificar sufijo2 si es necesario
* Cambiar datos por datos_xx si es necesario

In [18]:
# file_save <- paste0(caso, proceso, tarea, tcltk::tkgetSaveFile(), proper, extension) 
# path_out <- paste0(oPath, file_save)
# write_csv(data_to_save_xxxxx, path_out)

# cat('File saved as: ')
# path_out

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>

- Los ficheros de salida del proceso van siempre a Data/Output/.  

In [19]:
# file_save <- paste0(caso, proceso, tarea, archivo, proper, extension) 
# path_out <- paste0(oPath, file_save)
# write_csv(data_to_save_xxxxx, path_out)

# cat('File saved as: ')
# path_out

#### Copia del fichero a Input

Si el archivo se va a usar en otros notebooks, copiar a la carpeta Input

In [20]:
# path_in <- paste0(iPath, file_save)
# file.copy(path_out, path_in, overwrite = TRUE)

## <font color='#2874a6'>REPORT</font>

A continuación se realizará un informe de las acciones realizadas

## <font color=' #2874a6 '>Main Actions Carried Out</font>

- Si eran necesarias se han realizado en el proceso 05 por cuestiones de eficiencia
- O bien se hacen en el dominio IV o V para integrar en el pipeline de modelización

## <font color=' #2874a6 '>Main Conclusions</font>

- Los datos están listos para la modelización y despliegue


## <font color='RED'>CODE TO DEPLOY (PILOT)</font>

A continuación se incluirá el código que deba ser llevado a despliegue para producción, dado que se entiende efectúa operaciones necesarias sobre los datos en la ejecución del prototipo

<font color=' red '><b>Description</b></font>

- No hay nada que desplegar en el piloto, ya que estos datos son estáticos o en todo caso cambian con muy poca frecuencia, altamente improbable durante el proyecto.


<font color=' red '><b>CODE</b></font>